# Airbyte Salesforce (已弃用)

请注意：此连接器特定加载器已弃用。请改用 [`AirbyteLoader`](/docs/integrations/document_loaders/airbyte)。

>[Airbyte](https://github.com/airbytehq/airbyte) 是一个数据集成平台，用于从 API、数据库和文件到数据仓库和数据湖的 ELT 管道。它拥有最大的 ELT 连接器到数据仓库和数据库的目录。

此加载器公开了 Salesforce 连接器作为文档加载器，允许您将各种 Salesforce 对象加载为文档。

## 安装

首先，你需要安装 `airbyte-source-salesforce` 这个 Python 包。

In [ ]:
%pip install --upgrade --quiet  airbyte-source-salesforce

## 示例

请查阅 [Airbyte 文档页面](https://docs.airbyte.com/integrations/sources/salesforce/) 以了解 Reader 的配置详情。
Config 对象应遵循的 JSON Schema 可以在 Github 上找到：[https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-salesforce/source_salesforce/spec.yaml](https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-salesforce/source_salesforce/spec.yaml)。

大致结构如下：
```python
{
  "client_id": "<oauth client id>",
  "client_secret": "<oauth client secret>",
  "refresh_token": "<oauth refresh token>",
  "start_date": "<要从中开始检索记录的日期，ISO 格式，例如 2020-10-20T00:00:00Z>",
  "is_sandbox": False, # 如果您使用的是沙盒环境，请设为 True
  "streams_criteria": [ # 用于过滤应可加载的 Salesforce 对象的过滤器数组
    {"criteria": "exacts", "value": "Account"}, # Salesforce 对象的精确名称
    {"criteria": "starts with", "value": "Asset"}, # 名称前缀
    # 其他允许的条件：ends with, contains, starts not with, ends not with, not contains, not exacts
  ],
}
```

默认情况下，所有字段都将作为元数据存储在文档中，文本内容将设置为空字符串。通过转换 Reader 返回的文档来构建文档的文本内容。

In [ ]:
from langchain_community.document_loaders.airbyte import AirbyteSalesforceLoader

config = {
    # your salesforce configuration
}

loader = AirbyteSalesforceLoader(
    config=config, stream_name="asset"
)  # check the documentation linked above for a list of all streams

现在您可以像往常一样加载文档了

In [ ]:
docs = loader.load()

由于 `load` 返回一个列表，它会一直阻塞直到所有文档都被加载。为了对此过程有更好的控制，您也可以使用 `lazy_load` 方法，它会返回一个迭代器：

In [ ]:
docs_iterator = loader.lazy_load()

请注意，默认情况下页面内容为空，并且 metadata 对象包含记录中的所有信息。若要以其他方式创建文档，请在创建加载器时传入一个 `record_handler` 函数：

In [ ]:
from langchain_core.documents import Document


def handle_record(record, id):
    return Document(page_content=record.data["title"], metadata=record.data)


loader = AirbyteSalesforceLoader(
    config=config, record_handler=handle_record, stream_name="asset"
)
docs = loader.load()

## 增量加载

某些流支持增量加载，这意味着源会跟踪已同步的记录，并且不会再次加载它们。这对于数据量大且频繁更新的源非常有用。

要利用此功能，请存储加载器的 `last_state` 属性，并在再次创建加载器时将其传入。这将确保只加载新记录。

In [ ]:
last_state = loader.last_state  # store safely

incremental_loader = AirbyteSalesforceLoader(
    config=config, stream_name="asset", state=last_state
)

new_docs = incremental_loader.load()